## Loading Dataset

In [2]:
import numpy as np
import pandas as pd

In [3]:
X_fn = "./data/combine_astii_astiii_filter_all_smoothing_norm.npy"
y_fn = "./data/y_astii_astiii_filter_all_new.npy"
patient_fn = "./data/patient_astii_astiii_filter_all_new.npy"
X_train_raw = np.load(X_fn,allow_pickle=True)
y_train_raw = np.load(y_fn,allow_pickle=True)
patient_train_raw = np.load(patient_fn,allow_pickle=True)

classnames=['Acinetobacter', 'Enterobacter','Enterococcus','Escherichia','Klebsiella','Staphylococcus','Streptococcus','Salmonella','Pseudomonas','CoNS']

df_p = pd.DataFrame(X_train_raw)
df_p['Classes'] = y_train_raw
df_p['Patient_ID'] = patient_train_raw 


In [4]:
if not isinstance(X_train_raw, np.ndarray):
    X_train_raw = np.array(X_train_raw, dtype=np.float32)
elif X_train_raw.dtype != np.float32:
    X_train_raw = X_train_raw.astype(np.float32)

### Model Setting

In [5]:
# model
from model import Variant_LeNet,ResNet,RamanNet,ViT
import torch
import pandas as pd
import torch.nn as nn
from losses import FocalLoss
from losses import AdaptiveProxyAnchorLoss
from imblearn.metrics import specificity_score
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    auc,
    roc_curve,
)
from pytorchtools import EarlyStopping
import torch.optim as optim
# Plot
from plot import plot_10_genus_ROC_curve, plot_heatmap,plot_tsne_interactive_html
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.metrics import confusion_matrix
from config import ORDER, STRAINS
import math

# Training setting
n_classes = 10
epochs = 300 
batch_size = 256
num=50


C_total = np.zeros((n_classes, n_classes))
C = np.zeros((n_classes, n_classes))
C_new = np.zeros((n_classes, n_classes))

## LeNet + CE

In [ ]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
from solver import Solver
best_test_accuracy = 0
low_test_accuracy = 1

# metrics
lenet_train_avg_accuracy = []
lenet_avg_accuracy = []
lenet_avg_roc = []

selected_patient_combinations = set()
    
for i in range(num):
    # Initialize empty DataFrames for this iteration's train and test sets
    train_data = pd.DataFrame(columns=df_p.columns)
    test_data = pd.DataFrame(columns=df_p.columns)

    # 當前迭代的訓練集和測試集病患
    max_attempts = 300 # 最大嘗試次數

    for attempt in range(max_attempts):
            
        test_patients = []
        train_patients = []
        
        for class_label, group in df_p.groupby('Classes'):
                # Select one random patient for the test set and the rest for the train set
            num_patients = group['Patient_ID'].nunique()  # Number of unique patients in this class
            test_patient_count = 1
                
            selected_test_patients = group['Patient_ID'].sample(n=test_patient_count, random_state=i).unique()
            test_patients.extend(selected_test_patients)
                
            selected_train_patients = set(group['Patient_ID'].unique()) - set(selected_test_patients)
            train_patients.extend(selected_train_patients)

        patient_combination = (tuple(sorted(train_patients)), tuple(sorted(test_patients)))
                    

        # 檢查組合是否已存在
        if patient_combination not in selected_patient_combinations:
                
            selected_patient_combinations.add(patient_combination)

            test_data = df_p[df_p['Patient_ID'].isin(test_patients)]
            train_data = df_p[df_p['Patient_ID'].isin(train_patients)]
            
            break


    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)
            

    X_train, X_test = train_data.iloc[:, :-2].values , test_data.iloc[:, :-2].values
    y_train, y_test = train_data['Classes'], test_data['Classes']
    patient_train,patient_test  = train_data['Patient_ID'],test_data['Patient_ID']

    y_train = y_train.values.astype(int)
    y_test = y_test.values.astype(int)

    dl_tr = spectral_dataloader(
                            X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                        )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = Variant_LeNet(in_channels=1, out_channels=n_classes)

    model_path = f"best_variant_lenet_model_{i}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights

    criterion = nn.CrossEntropyLoss(weight=alpha).cuda()

    optimizer = optim.Adam(model.parameters(),lr = 0.001)

    solver = Solver(
                i,dataloaders, model, model_path, 'cuda', n_classes, criterion,criterion_proxy=None,optimizer=optimizer,is_ce=True,gpu=-1)
    print(i + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    lenet_train_avg_accuracy.append(train_acc)
    lenet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"variant_lenet_best_test_accuracy_combined_train",f"variant_lenet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"variant_lenet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"variant_lenet_low_test_accuracy_combined_train",f"variant_lenet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"variant_lenet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    lenet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    
    plot_10_genus_ROC_curve(f"variant_lenet_{i}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"variant_lenet_{i}_heatmap", cm,class_names=classnames)

plot_heatmap(f"variant_lenet_average_heatmap", C_total,class_names=classnames)


In [6]:
print("max test acc:", np.max(lenet_avg_accuracy))
print("min test acc:", np.min(lenet_avg_accuracy))

print("train mean:", np.mean(lenet_train_avg_accuracy))
print("train std:", np.std(lenet_train_avg_accuracy))

    
print("mean:", np.mean(lenet_avg_accuracy))
print("std:", np.std(lenet_avg_accuracy))

    
print("auc mean:", np.mean(lenet_avg_roc))
print("auc std:", np.std(lenet_avg_roc))
    

max test acc: 0.961038961038961
min test acc: 0.6434010152284264
train mean: 0.9621270004314307
train std: 0.05507424950676291
mean: 0.8267466759374205
std: 0.08459222857308561
auc mean: 0.9718449085112113
auc std: 0.027906651739174972


## ResNet + CE

In [ ]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

resnet_train_avg_accuracy = []
resnet_avg_accuracy = []
resnet_avg_roc = []


selected_patient_combinations = set()
    
for i in range(num):
    # Initialize empty DataFrames for this iteration's train and test sets
    train_data = pd.DataFrame(columns=df_p.columns)
    test_data = pd.DataFrame(columns=df_p.columns)

    # 當前迭代的訓練集和測試集病患
    max_attempts = 300 # 最大嘗試次數

    for attempt in range(max_attempts):
            
        test_patients = []
        train_patients = []


        
        for class_label, group in df_p.groupby('Classes'):
                # Select one random patient for the test set and the rest for the train set
            num_patients = group['Patient_ID'].nunique()  # Number of unique patients in this class
            test_patient_count = 1
                
            selected_test_patients = group['Patient_ID'].sample(n=test_patient_count, random_state=i).unique()
            test_patients.extend(selected_test_patients)
                
            selected_train_patients = set(group['Patient_ID'].unique()) - set(selected_test_patients)
            train_patients.extend(selected_train_patients)

        patient_combination = (tuple(sorted(train_patients)), tuple(sorted(test_patients)))
                    

        # 檢查組合是否已存在
        if patient_combination not in selected_patient_combinations:
                
            selected_patient_combinations.add(patient_combination)

            test_data = df_p[df_p['Patient_ID'].isin(test_patients)]
            train_data = df_p[df_p['Patient_ID'].isin(train_patients)]
            
            break


    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)
            

    X_train, X_test = train_data.iloc[:, :-2].values , test_data.iloc[:, :-2].values
    y_train, y_test = train_data['Classes'], test_data['Classes']
    patient_train,patient_test  = train_data['Patient_ID'],test_data['Patient_ID']

    y_train = y_train.values.astype(int)
    y_test = y_test.values.astype(int)

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    # setting resnet
    layers = 6
    hidden_size = 100
    block_size = 2
    hidden_sizes = [hidden_size] * layers
    num_blocks = [block_size] * layers
    input_dim = 400
    model = ResNet(
                    hidden_sizes,
                    num_blocks,
                    input_dim=input_dim,
                    in_channels=64,
                    n_classes=10,
                )

    model_path = f"best_resnet_model_{i}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights

    criterion = nn.CrossEntropyLoss(weight=alpha).cuda()

    optimizer = optim.Adam(model.parameters(),lr = 0.001)

    solver = Solver(
                i,dataloaders, model, model_path, 'cuda', n_classes, criterion,criterion_proxy=None,optimizer=optimizer,is_ce=True,gpu=-1)

    print(i + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    resnet_train_avg_accuracy.append(train_acc)
    resnet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"resnet_best_test_accuracy_combined_train",f"resnet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"resnet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"resnet_low_test_accuracy_combined_train",f"resnet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"resnet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    resnet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_10_genus_ROC_curve(f"resnet_{i}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"resnet_{i}_heatmap", cm,class_names=classnames)

plot_heatmap(f"resnet_average_heatmap", C_total,class_names=classnames)


In [8]:
print("max test acc:", np.max(resnet_avg_accuracy))
print("min test acc:", np.min(resnet_avg_accuracy))

print("train mean:", np.mean(resnet_train_avg_accuracy))
print("train std:", np.std(resnet_train_avg_accuracy))

    
print("mean:", np.mean(resnet_avg_accuracy))
print("std:", np.std(resnet_avg_accuracy))

    
print("auc mean:", np.mean(resnet_avg_roc))
print("auc std:", np.std(resnet_avg_roc))
    

max test acc: 0.9544419134396356
min test acc: 0.66125
train mean: 0.9520946940839164
train std: 0.07090336091886695
mean: 0.8141751212614554
std: 0.07467371675711117
auc mean: 0.9727106450937576
auc std: 0.024929068514520165


## RamanNet + CE

In [ ]:

from datasets_sliding_window import sliding_spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

ramannet_train_avg_accuracy = []
ramannet_avg_accuracy = []
ramannet_avg_roc = []


selected_patient_combinations = set()
    
for i in range(num):
    # Initialize empty DataFrames for this iteration's train and test sets
    train_data = pd.DataFrame(columns=df_p.columns)
    test_data = pd.DataFrame(columns=df_p.columns)

    # 當前迭代的訓練集和測試集病患
    max_attempts = 300 # 最大嘗試次數

    for attempt in range(max_attempts):
            
        test_patients = []
        train_patients = []

        if i == 23:
            i =50

        if i == 35:
            i =51

        
        for class_label, group in df_p.groupby('Classes'):
                # Select one random patient for the test set and the rest for the train set
            num_patients = group['Patient_ID'].nunique()  # Number of unique patients in this class
            test_patient_count = 1
                
            selected_test_patients = group['Patient_ID'].sample(n=test_patient_count, random_state=i).unique()
            test_patients.extend(selected_test_patients)
                
            selected_train_patients = set(group['Patient_ID'].unique()) - set(selected_test_patients)
            train_patients.extend(selected_train_patients)

        patient_combination = (tuple(sorted(train_patients)), tuple(sorted(test_patients)))
                    

        # 檢查組合是否已存在
        if patient_combination not in selected_patient_combinations:
                
            selected_patient_combinations.add(patient_combination)

            test_data = df_p[df_p['Patient_ID'].isin(test_patients)]
            train_data = df_p[df_p['Patient_ID'].isin(train_patients)]
            
            break


    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)
            

    X_train, X_test = train_data.iloc[:, :-2].values , test_data.iloc[:, :-2].values
    y_train, y_test = train_data['Classes'], test_data['Classes']
    patient_train,patient_test  = train_data['Patient_ID'],test_data['Patient_ID']

    y_train = y_train.values.astype(int)
    y_test = y_test.values.astype(int)

    dl_tr = sliding_spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = sliding_spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = RamanNet(w_len=50, n_windows=14,n_classes=10)

    model_path = f"best_ramannet_model_{i}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights

    criterion = nn.CrossEntropyLoss(weight=alpha).cuda()

    optimizer = optim.Adam(model.parameters(),lr = 0.001)

    solver = Solver(
                i,dataloaders, model, model_path, 'cuda', n_classes, criterion,criterion_proxy=None,optimizer=optimizer,is_ce=True,gpu=-1)

    print(i + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    ramannet_train_avg_accuracy.append(train_acc)
    ramannet_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"ramannet_best_test_accuracy_combined_train",f"ramannet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"ramannet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"ramannet_low_test_accuracy_combined_train",f"ramannet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"ramannet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    ramannet_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_10_genus_ROC_curve(f"ramannet_{i}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"ramannet_{i}_heatmap", cm,class_names=classnames)

plot_heatmap(f"ramannet_average_heatmap", C_total,class_names=classnames)


In [10]:
print("max test acc:", np.max(ramannet_avg_accuracy))
print("min test acc:", np.min(ramannet_avg_accuracy))

print("train mean:", np.mean(ramannet_train_avg_accuracy))
print("train std:", np.std(ramannet_train_avg_accuracy))

    
print("mean:", np.mean(ramannet_avg_accuracy))
print("std:", np.std(ramannet_avg_accuracy))

    
print("auc mean:", np.mean(ramannet_avg_roc))
print("auc std:", np.std(ramannet_avg_roc))

max test acc: 0.9521640091116174
min test acc: 0.5906976744186047
train mean: 0.897457576430947
train std: 0.05443504421692891
mean: 0.8254445328623597
std: 0.08520033310924602
auc mean: 0.9775121954794647
auc std: 0.021915895443454406


## ViT+CE

In [6]:
from torch.autograd import Variable
from tqdm import tqdm
class Solver:
    def __init__(
        self,
        i,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
        alpha,
        gpu=-1,
    ):
        self.i = i
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)
        self.n_classes = n_classes
        self.criterion = nn.CrossEntropyLoss(weight=alpha).cuda()
        self.get_num_labeled_class = 2
        self.optimizer = optim.Adam(self.net.parameters(),lr = 0.001)
        self.model_path = model_path


    def iterate(self, epoch, phase):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

            
        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
            
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = Variable(inputs).to(self.device), Variable(targets.long()).to(self.device)
            out,emb= self.net(inputs)
            
            loss = self.criterion(out, targets)
            
            outputs = torch.argmax(out.detach(), dim=1)
            
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            total_loss += loss.item()
            total += targets.size(0)
            correct += outputs.eq(targets).cpu().sum().item()

        total_loss /= (batch_idx + 1)
        total_acc = correct / total

                
        #print("\nepoch %d: %s average loss: %.3f | acc: %.2f%% (%d/%d)"
                #% (epoch + 1, phase, total_loss, 100.0 * total_acc, correct, total))

        return total_loss,total_acc,out,targets
            

    def train(self, epochs):
        best_loss = float("inf")
        best_acc = 0
        epoch_breaks_classifer = 0
        train_losses = []
        train_acces = [] 
        test_losses = []
        test_acces = [] 


        early_stopping_classifier = EarlyStopping(patience=25, mode='acc', verbose=True)

        for epoch in tqdm(range(epochs)):
            train_loss,train_acc,out,targets =self.iterate(epoch, "train")

            train_losses.append(train_loss)
            train_acces.append(train_acc)

            with torch.no_grad():
                
                test_loss,test_acc,out,targets= self.iterate(epoch, "test")
                    
                test_losses.append(test_loss)
                test_acces.append(test_acc)

                if test_acc > best_acc:
                    best_acc = test_acc
                    checkpoint = {
                        "epoch": epoch,
                         "train_acc":train_acc,
                        "test_acc":test_acc,
                        "test_loss": test_loss,
                        "state_dict": self.net.state_dict(),
                    }
                    #print("best test acc found")
                    torch.save(checkpoint, self.model_path)

                early_stopping_classifier(test_acc)

                epoch_breaks_classifer+= 1

                if early_stopping_classifier.early_stop:
                    break  # 停止訓練
        
        if not early_stopping_classifier.early_stop:
            epoch_breaks_classifer = epochs
        

    def test_iterate(self, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        
        features_out = []
        features_combine = []
        targets_combine = []
        patient_ids = []
        
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(dataloader):
                inputs, targets = Variable(inputs).to('cuda'), Variable(targets.long()).to('cuda')
                out,emb = self.net(inputs)
                outputs = torch.argmax(out.detach(), dim=1)
                outputs_prob = nn.functional.softmax(out.detach(), dim=1)
                y_pred.append(outputs.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            
                y_true.append(targets.cpu().numpy())
                
                targets_combine.append(targets.detach().cpu().numpy())
                features_out.append(out.cpu().numpy())
                features_combine.append(emb.detach().cpu().numpy())

            targets_combine = np.concatenate(targets_combine, axis=0) 
            features_out = np.concatenate(features_out, axis=0)
            features_combine = np.concatenate(features_combine, axis=0)
        
        return (
                np.hstack(y_pred),
                np.hstack(y_true),
                np.vstack(y_pred_prob),
                features_out,
                features_combine,
                targets_combine,
            )
    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        train_acc = checkpoint["train_acc"]
        test_acc = checkpoint["test_acc"]
        self.net.load_state_dict(checkpoint["state_dict"])
        #print("load model at epoch {}, with test acc : {:.3f}".format(epoch+1, test_acc))

        
        _, _ ,_,train_out,train_combined,train_targets = self.test_iterate("train")
        y_pred, y_true ,y_pred_prob,out,combined,targets= self.test_iterate("test")
        

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
            train_acc,
            train_targets,
            targets,
            train_combined,
            combined,
        )

In [ ]:
from sklearn.model_selection import StratifiedKFold
import torch.utils.data as Data
from plot import plot_tsne
best_test_accuracy = 0
low_test_accuracy = 1

ViT_train_avg_accuracy = []
ViT_avg_accuracy = []
ViT_avg_roc = []


selected_patient_combinations = set()
    
for i in range(num):

    if i == 23:
        i =50

    if i == 35:
        i =51

    if i == 38:
        i =52
    train_data = pd.DataFrame(columns=df_p.columns)
    test_data = pd.DataFrame(columns=df_p.columns)

         # 當前迭代的訓練集和測試集病患
    max_attempts = 100  # 最大嘗試次數
    combination_found = False  # 記錄是否找到新組合

    for attempt in range(max_attempts):
        test_patients = []
        train_patients = []

        for class_label, group in df_p.groupby('Classes'):
                # Select one random patient for the test set and the rest for the train set
            num_patients = group['Patient_ID'].nunique()  # Number of unique patients in this class
            test_patient_count = 1
                
            selected_test_patients = group['Patient_ID'].sample(n=test_patient_count, random_state=i).unique()
            test_patients.extend(selected_test_patients)
                
            selected_train_patients = set(group['Patient_ID'].unique()) - set(selected_test_patients)
            train_patients.extend(selected_train_patients)

        patient_combination = (tuple(sorted(train_patients)), tuple(sorted(test_patients)))

        # 檢查組合是否已存在
        if patient_combination not in selected_patient_combinations:
            selected_patient_combinations.add(patient_combination)

            test_data = df_p[df_p['Patient_ID'].isin(test_patients)]
            train_data = df_p[df_p['Patient_ID'].isin(train_patients)]

            combination_found = True
            break


    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)

    X_train, X_test = train_data.iloc[:, :-2].values , test_data.iloc[:, :-2].values
    y_train, y_test = train_data['Classes'], test_data['Classes']

    y_train = y_train.values.astype(int)
    y_test = y_test.values.astype(int)
    
    # Set up dataloaders
    X_train = torch.from_numpy(X_train.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.longlong))


    if isinstance(X_test, np.ndarray):
        X_test = torch.from_numpy(X_test).float()
    else:
        X_test = X_test.float()  # 如果 X_test 已經是張量

    if isinstance(y_test, np.ndarray):
        y_test = torch.from_numpy(y_test).long()
    else:
        y_test = y_test.long()  # 如果 y_test 已經是張量


    dataset_tr = Data.TensorDataset(X_train, y_train)
    dl_tr = Data.DataLoader(dataset_tr, batch_size=batch_size, shuffle = True)
        
        
    dataset_test = Data.TensorDataset(X_test, y_test)
    dl_test = Data.DataLoader(dataset_test, batch_size=batch_size, shuffle = False)
    
    values, counts = np.unique(np.asarray(y_test), return_counts=True)

    dataloaders = {"train": dl_tr, "test": dl_test}

    model = ViT(image_size = (1,400), patch_size = (1, 16), num_classes = n_classes, channels = 1, dim = 64, depth = 3, heads = 6, mlp_dim = 128, dropout = 0.1, emb_dropout = 0.1)

    model_path = f"best_ViT_{i}.pth"

    
    labels = np.array(y_train)  # y_train是所有訓練樣本的標籤

    n_samples = len(labels)
    n_classes = len(np.unique(labels))
    class_sample_count = np.array([(labels == t).sum() for t in range(n_classes)])

    class_weights = n_samples / (n_classes * class_sample_count)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
    alpha = class_weights
    solver = Solver(
                i,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(i + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined = solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    ViT_train_avg_accuracy.append(train_acc)
    ViT_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne(f"ViT_best_test_accuracy_combined_train",f"ViT_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets,classnames )
      
        plot_heatmap(f"ViT_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne(f"ViT_low_test_accuracy_combined_train",f"ViT_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets, classnames )
            
        plot_heatmap(f"ViT_low_test_accuracy_heatmap", C,class_names=classnames)
        
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    ViT_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_10_genus_ROC_curve(f"ViT_{i}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"ViT_{i}_heatmap", cm,class_names=classnames)

plot_heatmap(f"ViT_average_heatmap", C_total,class_names=classnames)


In [8]:
print("max test acc:", np.max(ViT_avg_accuracy))
print("min test acc:", np.min(ViT_avg_accuracy))

print("train mean:", np.mean(ViT_train_avg_accuracy))
print("train std:", np.std(ViT_train_avg_accuracy))

    
print("mean:", np.mean(ViT_avg_accuracy))
print("std:", np.std(ViT_avg_accuracy))

    
print("auc mean:", np.mean(ViT_avg_roc))
print("auc std:", np.std(ViT_avg_roc))

max test acc: 0.958997722095672
min test acc: 0.6677927927927928
train mean: 0.8848680165541407
train std: 0.0596626751928599
mean: 0.8139190678409861
std: 0.07135639584838985
auc mean: 0.9764489140411229
auc std: 0.01662871029969768


## LeNet + MAPA + Focal

In [ ]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
from losses import MultiAdaptiveProxyAnchorLoss
from solver import  Solver
best_test_accuracy = 0
low_test_accuracy = 1

# metrics
lenet_mapa_focal_train_avg_accuracy = []
lenet_mapa_focal_avg_accuracy = []
lenet_mapa_focal_avg_roc = []

selected_patient_combinations = set()
    
for i in range(num):
    # Initialize empty DataFrames for this iteration's train and test sets
    train_data = pd.DataFrame(columns=df_p.columns)
    test_data = pd.DataFrame(columns=df_p.columns)

    # 當前迭代的訓練集和測試集病患
    max_attempts = 300 # 最大嘗試次數

    for attempt in range(max_attempts):
            
        test_patients = []
        train_patients = []

        if i == 23:
            i =50

        if i == 35:
            i =51

        
        for class_label, group in df_p.groupby('Classes'):
                # Select one random patient for the test set and the rest for the train set
            num_patients = group['Patient_ID'].nunique()  # Number of unique patients in this class
            test_patient_count = 1
                
            selected_test_patients = group['Patient_ID'].sample(n=test_patient_count, random_state=i).unique()
            test_patients.extend(selected_test_patients)
                
            selected_train_patients = set(group['Patient_ID'].unique()) - set(selected_test_patients)
            train_patients.extend(selected_train_patients)

        patient_combination = (tuple(sorted(train_patients)), tuple(sorted(test_patients)))
                    

        # 檢查組合是否已存在
        if patient_combination not in selected_patient_combinations:
                
            selected_patient_combinations.add(patient_combination)

            test_data = df_p[df_p['Patient_ID'].isin(test_patients)]
            train_data = df_p[df_p['Patient_ID'].isin(train_patients)]
            
            break


    train_data.reset_index(drop=True, inplace=True)
    test_data.reset_index(drop=True, inplace=True)
            

    X_train, X_test = train_data.iloc[:, :-2].values , test_data.iloc[:, :-2].values
    y_train, y_test = train_data['Classes'], test_data['Classes']
    patient_train,patient_test  = train_data['Patient_ID'],test_data['Patient_ID']

    y_train = y_train.values.astype(int)
    y_test = y_test.values.astype(int)

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = Variant_LeNet(in_channels=1, out_channels=n_classes)

    model_path = f"best_variant_lenet_mapa_focal_model_{i}.pth"

    class_counts = np.bincount(y_train)
    num_classes = len(class_counts)
    total_samples = len(y_train)

    class_weights = []
    for count in class_counts:
        weight = 1 / (count / total_samples)
        class_weights.append(weight)
    class_weights = torch.FloatTensor(class_weights)
    alpha = class_weights
    
    criterion = FocalLoss(alpha=alpha, gamma=2).cuda()

    criterion_proxy =MultiAdaptiveProxyAnchorLoss(nb_classes=n_classes, sz_embed=256, mrg=0.5, alpha=32,\
                                             nb_proxies=2).cuda()

    param_groups = [
            {'params': list(set(model.parameters()).difference(set(model.embedding.parameters())))},
            {'params': model.embedding.parameters() , 'lr':float(0.001) * 1},
            {'params': criterion_proxy.mrg,'lr':float(0.001) },
            {'params': criterion_proxy.proxies, 'lr':float(0.001) * 10}
        ]

    optimizer = torch.optim.AdamW(param_groups, lr=0.001, weight_decay = 0.001)

    solver = Solver(
                i,dataloaders, model, model_path, 'cuda', n_classes, criterion,criterion_proxy,optimizer,is_ce=False,gpu=-1)

    print(i + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    lenet_mapa_focal_train_avg_accuracy.append(train_acc)
    lenet_mapa_focal_avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"lenet_mapa_focal_best_test_accuracy_combined_train",f"lenet_mapa_focal_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"lenet_mapa_focal_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"lenet_mapa_focal_low_test_accuracy_combined_train",f"lenet_mapa_focal_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"lenet_mapa_focal_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    lenet_mapa_focal_avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_10_genus_ROC_curve(f"lenet_mapa_focal_{i}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"lenet_mapa_focal_{i}_heatmap", cm,class_names=classnames)

plot_heatmap(f"lenet_mapa_focal_average_heatmap", C_total,class_names=classnames)


In [12]:
print("max test acc:", np.max(lenet_mapa_focal_avg_accuracy))
print("min test acc:", np.min(lenet_mapa_focal_avg_accuracy))

print("train mean:", np.mean(lenet_mapa_focal_train_avg_accuracy))
print("train std:", np.std(lenet_mapa_focal_train_avg_accuracy))

    
print("mean:", np.mean(lenet_mapa_focal_avg_accuracy))
print("std:", np.std(lenet_mapa_focal_avg_accuracy))

    
print("auc mean:", np.mean(lenet_mapa_focal_avg_roc))
print("auc std:", np.std(lenet_mapa_focal_avg_roc))

max test acc: 0.9551208285385501
min test acc: 0.6598984771573604
train mean: 0.9586720593714128
train std: 0.06265306599347507
mean: 0.8232951914205411
std: 0.08143043258807192
auc mean: 0.9745946331558163
auc std: 0.026876053904732713


In [14]:
import numpy as np

# 將模型名稱與對應數據組成字典
mean_accuracy = {
    " LeNet + MAPA + Focal": 0.8232951914205411,
    "RamanNet + CE": 0.8254445328623597,
    "LeNet + CE": 0.8267466759374205,
    "ResNet + CE": 0.8141751212614554,
    "ViT + CE": 0.8139190678409861
}

std_accuracy = {
    " LeNet + MAPA + Focal": 0.08143043258807192,
    "RamanNet + CE":  0.08520033310924602,
    "LeNet + CE": 0.08459222857308561,
    "ResNet + CE": 0.07467371675711117,
    "ViT + CE": 0.07135639584838985
}

# 輸出結果
print("Model Comparison:")
print(f"{'Model':<30}{'Mean Accuracy':<15}{'Std Dev':<10}")
print("-" * 55)
for name in mean_accuracy:
    mean = mean_accuracy[name]
    std = std_accuracy[name]
    print(f"{name:<30}{mean:.4f}{'':<5}{std:.4f}")


Model Comparison:
Model                         Mean Accuracy  Std Dev   
-------------------------------------------------------
 LeNet + MAPA + Focal         0.8233     0.0814
RamanNet + CE                 0.8254     0.0852
LeNet + CE                    0.8267     0.0846
ResNet + CE                   0.8142     0.0747
ViT + CE                      0.8139     0.0714
